In [1]:
#!/usr/bin/env python
# coding: utf-8

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
4,application_1691410809424_0005,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import os
import sys
import pytz
import numpy as np
import datetime
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf, lpad, translate
from datetime import datetime 
from datetime import timedelta
from datetime import date
from dateutil.relativedelta import relativedelta
from pyspark.sql.types import *
from pyspark.sql.functions import count, avg

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
agora=datetime.now(pytz.timezone('America/Sao_Paulo'))
dthproc=agora.strftime("%Y%m%d%H%M%S")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
spark = SparkSession.builder.appName('trusted_HomeCredit_columns_description_20230805').getOrCreate()
sqlContext=SQLContext(spark.sparkContext)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
_DIA_AULA_ = '20230804'
_REF_ = '202001'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
raw ='t_HomeCredit_columns_description_20230805'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
bucket_trusted = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/003_trusted'.format(dia=_DIA_AULA_)
bucket_raw = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/002_raw'.format(dia=_DIA_AULA_)
bucket_control = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/005_control'.format(dia=_DIA_AULA_)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
output_trusted = 'trusted_HomeCredit_columns_description_20230805'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
full_path_raw = 's3://{bkt}/{file}'.format(bkt=bucket_raw,file=raw)
print(full_path_raw)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://bkt-datalake-grupo-01-hackathon-podacademy/20230804/002_raw/t_HomeCredit_columns_description_20230805

In [10]:
path_lake = os.path.join(full_path_raw)
raw_00=sqlContext.read.format("parquet").load(path_lake)
raw_00.registerTempTable("raw_00")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
raw_00.show(5,truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+------------------+--------------+---+----------------------------+------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+-------------+----------------------------+-----------------+
|ref   |ts_file_generation|ts_proc       |_c0|Table                       |Row               |Description                                                                                                                                                                                |Special|ref_partition|ts_file_generation_partition|ts_proc_partition|
+------+------------------+--------------+---+----------------------------+------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+------

In [27]:
# +------+------------------+--------------+---+----------------------------+------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+-------------+----------------------------+-----------------+
# |ref   |ts_file_generation|ts_proc       |_c0|Table                       |Row               |Description                                                                                                                                                                                |Special|ref_partition|ts_file_generation_partition|ts_proc_partition|
# +------+------------------+--------------+---+----------------------------+------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+-------------+----------------------------+-----------------+
# |202308|2023080500        |20230806155654|1  |application_{train|test}.csv|SK_ID_CURR        |ID of loan in our sample                                                                                                                                                                   |null   |202308       |2023080500                  |20230806155654   |
# |202308|2023080500        |20230806155654|2  |application_{train|test}.csv|TARGET            |Target variable (1 - client with payment difficulties: he/she had late payment more than X days on at least one of the first Y installments of the loan in our sample, 0 - all other cases)|null   |202308       |2023080500                  |20230806155654   |
# |202308|2023080500        |20230806155654|5  |application_{train|test}.csv|NAME_CONTRACT_TYPE|Identification if loan is cash or revolving                                                                                                                                                |null   |202308       |2023080500                  |20230806155654   |
# |202308|2023080500        |20230806155654|6  |application_{train|test}.csv|CODE_GENDER       |Gender of the client                                                                                                                                                                       |null   |202308       |2023080500                  |20230806155654   |
# |202308|2023080500        |20230806155654|7  |application_{train|test}.csv|FLAG_OWN_CAR      |Flag if the client owns a car                                                                                                                                                              |null   |202308       |2023080500                  |20230806155654   |
# +------+------------------+--------------+---+----------------------------+------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+-------------+----------------------------+-----------------+
# only showing top 5 rows

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
print('lista de colunas para tipar')
for col in raw_00.columns:
    print('cast(' + col + ' as) as ' + col + ',')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

lista de colunas para tipar
cast(ref as) as ref,
cast(ts_file_generation as) as ts_file_generation,
cast(ts_proc as) as ts_proc,
cast(_c0 as) as _c0,
cast(Table as) as Table,
cast(Row as) as Row,
cast(Description as) as Description,
cast(Special as) as Special,
cast(ref_partition as) as ref_partition,
cast(ts_file_generation_partition as) as ts_file_generation_partition,
cast(ts_proc_partition as) as ts_proc_partition,

In [14]:
lake = spark.sql(     
    """
        select
            -- padrao para todas as cargas no datalake--
            
            ref,
            ref_partition,
            
            {pdthproc} as ts_proc,
            {pdthproc} as ts_proc_partition,
        
            -- campos do arquivo --

            cast(_c0 as string) as _c0,
            cast(Table as string) as Table,
            cast(Row as string) as Row,
            cast(Description as string) as Description,
            cast(Special as string) as Special
            
        from
            raw_00
        --where
            --ref_partition = {_ref_}
        --    ref = {_ref_}
            
            
    """.format(pdthproc=dthproc,_ref_ = _REF_))
lake.registerTempTable("lake")
lake.cache()
lake.count()  

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

219

In [15]:
lake.explain()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
InMemoryTableScan [ref#0, ref_partition#8, ts_proc#56L, ts_proc_partition#57L, _c0#58, Table#59, Row#60, Description#61, Special#62]
   +- InMemoryRelation [ref#0, ref_partition#8, ts_proc#56L, ts_proc_partition#57L, _c0#58, Table#59, Row#60, Description#61, Special#62], StorageLevel(disk, memory, deserialized, 1 replicas)
         +- *(1) Project [ref#0, ref_partition#8, 20230807095150 AS ts_proc#56L, 20230807095150 AS ts_proc_partition#57L, _c0#3, Table#4, Row#5, Description#6, Special#7]
            +- *(1) FileScan parquet [ref#0,_c0#3,Table#4,Row#5,Description#6,Special#7,ref_partition#8,ts_file_generation_partition#9,ts_proc_partition#10L] Batched: true, Format: Parquet, Location: InMemoryFileIndex[s3://bkt-datalake-grupo-01-hackathon-podacademy/20230804/002_raw/t_HomeCredit_c..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<ref:int,_c0:string,Table:string,Row:string,Description:string,Special:string>

In [16]:
lake.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ref: integer (nullable = true)
 |-- ref_partition: integer (nullable = true)
 |-- ts_proc: long (nullable = false)
 |-- ts_proc_partition: long (nullable = false)
 |-- _c0: string (nullable = true)
 |-- Table: string (nullable = true)
 |-- Row: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Special: string (nullable = true)

In [17]:
lake.show(4,truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------------+--------------+-----------------+---+----------------------------+------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+
|ref   |ref_partition|ts_proc       |ts_proc_partition|_c0|Table                       |Row               |Description                                                                                                                                                                                |Special|
+------+-------------+--------------+-----------------+---+----------------------------+------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+
|202308|202308       |20230807095150|20230807095150   |1  |application_{train|test}.csv|

In [25]:
# +------+-------------+--------------+-----------------+---+----------------------------+------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+
# |ref   |ref_partition|ts_proc       |ts_proc_partition|_c0|Table                       |Row               |Description                                                                                                                                                                                |Special|
# +------+-------------+--------------+-----------------+---+----------------------------+------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+
# |202308|202308       |20230807095150|20230807095150   |1  |application_{train|test}.csv|SK_ID_CURR        |ID of loan in our sample                                                                                                                                                                   |null   |
# |202308|202308       |20230807095150|20230807095150   |2  |application_{train|test}.csv|TARGET            |Target variable (1 - client with payment difficulties: he/she had late payment more than X days on at least one of the first Y installments of the loan in our sample, 0 - all other cases)|null   |
# |202308|202308       |20230807095150|20230807095150   |5  |application_{train|test}.csv|NAME_CONTRACT_TYPE|Identification if loan is cash or revolving                                                                                                                                                |null   |
# |202308|202308       |20230807095150|20230807095150   |6  |application_{train|test}.csv|CODE_GENDER       |Gender of the client                                                                                                                                                                       |null   |
# +------+-------------+--------------+-----------------+---+----------------------------+------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+
# only showing top 4 rows

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
path_trusted = os.path.join('s3://',bucket_trusted,output_trusted)
print(path_trusted)

lake.coalesce(1).write. \
partitionBy("ref_partition","ts_proc_partition"). \
parquet(path_trusted,mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://bkt-datalake-grupo-01-hackathon-podacademy/20230804/003_trusted/trusted_HomeCredit_columns_description_20230805

In [19]:
varDataFile = output_trusted

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
controle = spark.sql(    
        """
            select
                '{tb}' as name_file,
                '{tb}' as name_file_partition,
                
                ref,
                ref_partition,
                
                ts_proc,
                ts_proc_partition,
                
                count(*) as qtd_registros
            from 
                lake as a
            group by
                1,2,3,4,5,6
            order by
                1,2,3,4,5,6
        """.format(tb=varDataFile))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
controle.registerTempTable("controle")
controle.cache()
qtd=controle.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
controle.show(qtd,truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------------------------+-----------------------------------------------+------+-------------+--------------+-----------------+-------------+
|name_file                                      |name_file_partition                            |ref   |ref_partition|ts_proc       |ts_proc_partition|qtd_registros|
+-----------------------------------------------+-----------------------------------------------+------+-------------+--------------+-----------------+-------------+
|trusted_HomeCredit_columns_description_20230805|trusted_HomeCredit_columns_description_20230805|202308|202308       |20230807095150|20230807095150   |219          |
+-----------------------------------------------+-----------------------------------------------+------+-------------+--------------+-----------------+-------------+

In [24]:
# +-----------------------------------------------+-----------------------------------------------+------+-------------+--------------+-----------------+-------------+
# |name_file                                      |name_file_partition                            |ref   |ref_partition|ts_proc       |ts_proc_partition|qtd_registros|
# +-----------------------------------------------+-----------------------------------------------+------+-------------+--------------+-----------------+-------------+
# |trusted_HomeCredit_columns_description_20230805|trusted_HomeCredit_columns_description_20230805|202308|202308       |20230807095150|20230807095150   |219          |
# +-----------------------------------------------+-----------------------------------------------+------+-------------+--------------+-----------------+-------------+

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
path_control = os.path.join('s3://',bucket_control,'tb_0002_controle_procesamento_trusted')
print(path_control)
controle.coalesce(1).write.\
parquet(path_control,mode='append')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://bkt-datalake-grupo-01-hackathon-podacademy/20230804/005_control/tb_0002_controle_procesamento_trusted